In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import itertools
import random
import string
import io
import warnings

Details about the model and character set used in pet names

In [2]:
character_list = list(string.ascii_lowercase) + [".","-"," ","+"]
character_lookup = dict(zip(character_list, range(len(character_list))))
max_length = 10
num_characters = len(character_lookup)

Load the model that was trained in `train_model.ipynb`:

In [3]:
model = keras.models.load_model("model.h5")

This function generates a name using the trained model. It creates on character at a time and then reruns the model on the partially compelete name. A value is sampled from the distribution that is outputted by the model, with a "temperature" factor that makes the names more or less chaotic.

In [10]:
def generate_name(model, character_lookup, max_length, temperature = 1):
    """Generate a pet name
    This function takes the model and parameters then generates a new name from it.
    """
    def choose_next_char(preds, character_lookup, temperature = 1):
        """Given a distribution of character probabilities, returns a character sampled from it."""
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="divide by zero encountered in log")
            preds = np.log(preds)/temperature
        exp_preds = np.exp(preds)
        preds = exp_preds/sum(np.exp(preds))
        preds = preds.astype('float64')
        preds = preds / np.sum(preds)

        next_index = np.argmax(np.random.multinomial(1,preds))
        next_character = list(character_lookup.keys())[next_index]
        return next_character
    def characters_to_matrix(character_data):
        """Takes a list of partial names and converts them to a matrix"""
        character_data = [[character_lookup[chr] for chr in c] for c in character_data]
        padded_character_data = keras.utils.pad_sequences(character_data, maxlen = max_length)
        text_matrix = keras.utils.to_categorical(padded_character_data, num_classes = num_characters)
        return text_matrix

    in_progress_name = []
    next_letter = ""
  
    # While we haven't hit a stop character or exceeded some name length limit.
    while next_letter != "+" and len(in_progress_name) < 30:
        previous_letters_data = characters_to_matrix([in_progress_name])
        next_letter_probabilities = model.predict(previous_letters_data, verbose = 0)[0]
        next_letter = choose_next_char(next_letter_probabilities, character_lookup, temperature)
        if next_letter != "+":
            in_progress_name = in_progress_name + [next_letter]
    
    raw_name = ''.join(in_progress_name)
    capitalized_name = string.capwords(raw_name) 
  
    return capitalized_name

def generate_many_names(model, character_lookup, max_length, temperature = 1, n = 10):
    """Generate many names at once rather than one."""
    names = [generate_name(model, character_lookup, max_length, temperature) for i in range(n)]
    return names

An example of running the function:

In [17]:
names = generate_many_names(model, character_lookup, max_length, n = 20, temperature = 0.85)
", ".join(names)

'Wilgca, Havy, Rosa, Nicho, Ellie, Amaggie, Abuddy, Jake, Beegey, Deo, Blio, Lailey, Pati, Carley, Millie, Dedgi, Lulu, Mil, Sidiue, Bailey'